In [78]:
import pandas as pd
import json
import plotly.graph_objects as go
import numpy as np 

# Load and preprocess data
def _load_and_preprocess_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    compositions = []
    for item in data:
        comp = item['Composition']
        # Assuming each phase dict has a single key, get the first key as the dominant phase
        dominant_phase = list(item['Phases'].keys())[0]
        comp['Phase'] = dominant_phase
        compositions.append(comp)
    return pd.DataFrame(compositions)
import pandas as pd
import json
import json
import pandas as pd

def load_and_preprocess_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)

    # Prepare lists for DataFrame construction
    V = []
    Cr = []
    Ti = []
    Phases = []

    # Iterate through each record in the JSON data
    for key, item in data.items():
        composition = item['Composition']
        phases = item['Phases']

        # Append data to lists
        V.append(composition['V'])
        Cr.append(composition['Cr'])
        Ti.append(composition['Ti'])

        # Check if multiple phases are present with significant amounts
        significant_phases = {phase: value for phase, value in phases.items() if value > 0.1}
        if len(significant_phases) > 1:
            Phases.append('grey')  # Mark as grey if multiple significant phases
        else:
            dominant_phase = max(significant_phases, key=significant_phases.get, default='grey')
            Phases.append(dominant_phase)

    # Create DataFrame
    df = pd.DataFrame({
        'V': V,
        'Cr': Cr,
        'Ti': Ti,
        'Phases': Phases
    })

    return df


def plot_activation_ternary(df, threshold=True, min_log = None, max_log = None):
    if threshold:
        # Use the categorical 'color' column from df
        marker_colors = df['color']
        showscale = False
    else:
        # Compute the log10 values of time_to_threshold
        log_time_to_threshold = np.log10(df['time_to_threshold'])
        if min_log is None:
            min_log = np.floor(log_time_to_threshold.min())
        if max_log is None:
            max_log = np.ceil(log_time_to_threshold.max())
        #min_log = np.floor(log_time_to_threshold.min())  # Floor the min value
        #max_log = np.ceil(log_time_to_threshold.max())   # Ceil the max value

        # Set the marker colors directly to log values for continuous scale
        marker_colors = log_time_to_threshold
        showscale = True  # Enable color scale for heatmap

    # Create ternary plot with appropriate settings
    fig = go.Figure(go.Scatterternary({
        'mode': 'markers',
        'a': df['V'],
        'b': df['Cr'],
        'c': df['Ti'],
        'marker': {
            'symbol': 100,
            'color': marker_colors,
            'cmin': min_log if not threshold else None,
            'cmax': max_log if not threshold else None,
            'colorscale': 'RdYlGn_r',  # Red to green reversed scale
            'colorbar': {'title': 'Log10(Time to Threshold)'} if showscale else None,
            'size': 5,
            'line': {'width': 2}
        }
    }))

    fig.update_layout({
        'ternary': {
            'sum': 1,
            'aaxis': {'title': 'V', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'},
            'baxis': {'title': 'Cr', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'},
            'caxis': {'title': 'Ti', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'}
        },
        #'title': 'Ternary Plot of Compositions'
    })
    
    return fig

import plotly.graph_objects as go

def plot_phase_ternary(df, color_phase=True):
    if color_phase:
        # Define a mapping from phases to colors
        phase_colors = {
            'BCC_B2#2': 'yellow',
            'C15_LAVES#1': 'yellow',
            'BCC_B2#1': 'green',
            'HCP_A3#1': 'yellow',
            'Multiple': 'yellow'  # Add this line
        }
        # Map phases to colors
        marker_colors = df['Phases'].map(phase_colors).fillna('yellow')  # Use grey for unmapped phases or multiple phases
    else:
        marker_colors = 'grey'  # Default color if not using phase colors

    fig = go.Figure(go.Scatterternary({
        'mode': 'markers',
        'a': df['V'],
        'b': df['Cr'],
        'c': df['Ti'],
        'marker': {
            'symbol': 100,
            'color': marker_colors,
            'size': 5,
            'line': {'width': 2}
        }
    }))

    fig.update_layout({
        'ternary': {
            'sum': 1,
            'aaxis': {'title': 'V', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'},
            'baxis': {'title': 'Cr', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'},
            'caxis': {'title': 'Ti', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'}
        },
        #'title': 'Ternary Plot of Compositions with Phases'
    })
    
    return fig


In [89]:
activation_df = load_data('ternary_plot_data.json')
constrained_df = activation_df[activation_df['V'] >= 0.85]
fig = plot_activation_ternary(constrained_df, threshold=True)

fig.show()
fig.write_image('ternary_dbtt.png')

In [67]:
# Load the data
df = load_and_preprocess_data('all_results.json')

# print all the unique phases, the phases are in df['Phases']
for phase in df['Phases'].unique():
    print(phase)

HCP_A3#1
grey
C15_LAVES#1
BCC_B2#2
BCC_B2#1


In [68]:
# Plot
fig = plot_phase_ternary(df)
fig.show()
fig.write_image('thermo_dynamics_plot.png')


In [79]:
import json

def transform_results(input_file, output_file):
    with open(input_file, 'r') as file:
        data = json.load(file)

    transformed_data = []
    # Transform data
    for item in data:
        entry = item['Composition']  # Assumes this is already a dictionary with 'V', 'Cr', 'Ti'
        entry['Phases'] = item['Phases']  # Add phases
        # Determine color based on phase presence
        if 'BCC_B2#1' in item['Phases'] and len(item['Phases']) == 1:
            entry['color'] = 'darkgreen'
        else:
            entry['color'] = 'yellow'
        transformed_data.append(entry)

    # Save transformed data to a new JSON file
    with open(output_file, 'w') as outfile:
        json.dump(transformed_data, outfile)

    return output_file  # Return the path of the new file

# Example usage
output_file = transform_results('fixed_all_results.json', 'transformed_results.json')
print(f"Transformed results saved to: {output_file}")


Transformed results saved to: transformed_results.json


In [82]:
def load_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return pd.DataFrame(data)


trans_df = load_data('transformed_results.json')
#print(trans_df.head())

activation_df = load_data('ternary_plot_data.json')
#print(activation_df.head())

# Check the columns of each DataFrame
print("Columns in trans_df:", trans_df.columns)
print("Columns in activation_df:", activation_df.columns)


Columns in trans_df: Index(['V', 'Cr', 'Ti', 'Phases', 'color'], dtype='object')
Columns in activation_df: Index(['V', 'Cr', 'Ti', 'color'], dtype='object')


In [85]:
import plotly.graph_objects as go

# Merge the dataframes on the V, Cr, and Ti columns
merged_df = pd.merge(trans_df, activation_df, on=['V', 'Cr', 'Ti'], suffixes=('_trans', '_act'))

# Determine the final color based on the intersection logic
merged_df['color'] = merged_df.apply(
    lambda row: 'darkgreen' if row['color_trans'] == 'darkgreen' and row['color_act'] == 'darkgreen' else 'yellow',
    axis=1
)

# Prepare for plotting the ternary diagram
def plot_ternary(df):
    fig = go.Figure(go.Scatterternary({
        'mode': 'markers',
        'a': df['V'],
        'b': df['Cr'],
        'c': df['Ti'],
        'marker': {
            'symbol': 100,
            'color': df['color'],
            'size': 5,
            'line': {'width': 2}
        }
    }))

    fig.update_layout({
        'ternary': {
            'sum': 1,
            'aaxis': {'title': 'V', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'},
            'baxis': {'title': 'Cr', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'},
            'caxis': {'title': 'Ti', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'}
        },
        #'title': 'Ternary Plot of Compositions'
    })
    
    return fig

# Plot the ternary diagram
fig = plot_ternary(merged_df)
fig.show()
fig.write_image('therm_act_combined_plot.png')


In [86]:
import plotly.graph_objects as go

def plot_ternary_with_vanadium_constraint(df):
    # Filter the dataframe for compositions with 85% or more vanadium
    constrained_df = df[df['V'] >= 0.85]
    
    # Create the ternary plot using the filtered data
    fig = go.Figure(go.Scatterternary({
        'mode': 'markers',
        'a': constrained_df['V'],
        'b': constrained_df['Cr'],
        'c': constrained_df['Ti'],
        'marker': {
            'symbol': 100,
            'color': constrained_df['color'],
            'size': 5,  # Adjust size as needed
            'line': {'width': 2}
        }
    }))
    
    fig.update_layout({
        'ternary': {
            'sum': 1,
            'aaxis': {'title': 'V', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'},
            'baxis': {'title': 'Cr', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'},
            'caxis': {'title': 'Ti', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'}
        },
        'title': 'Ternary Plot of Compositions with ≥85% Vanadium'
    })
    
    return fig

# Assuming 'merged_df' is your dataframe from previous steps
fig = plot_ternary_with_vanadium_constraint(merged_df)
fig.show()
